In [ ]:
!pip install pydub
!pip install librosa tensorflow
!pip install numpy tensorflow
!pip install gtts


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, regularizers, optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pydub import AudioSegment
from gtts import gTTS
from pydub import AudioSegment
import librosa

# Your specific imports
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
#.MP3 to .WAV files
# Specify the source directory containing your MP3 files
source_directory = "/content/drive/MyDrive/Recordings_MP3/hindi"

# Specify the destination directory for the WAV files
destination_directory = "/content/drive/MyDrive/WAV_Files_Recordings/hindi"

# Specify the file to keep track of converted files
converted_file_path = "/content/drive/MyDrive/TXT_Files_AccentConversions/converted_files.txt"

# Ensure the destination directory exists, or create it
os.makedirs(destination_directory, exist_ok=True)

# Initialize a set to keep track of converted filenames
converted_files = set()

# Check if the converted file tracking exists and load the converted filenames
if os.path.exists(converted_file_path):
    with open(converted_file_path, 'r') as file:
        converted_files = set(file.read().splitlines())

# Loop through the MP3 files in the source directory and convert them to WAV if not already converted
for mp3_file in os.listdir(source_directory):
    if mp3_file.endswith(".mp3") and mp3_file not in converted_files:
        mp3_path = os.path.join(source_directory, mp3_file)
        wav_file = os.path.splitext(mp3_file)[0] + ".wav"
        wav_path = os.path.join(destination_directory, wav_file)

        # Convert MP3 to WAV using pydub
        sound = AudioSegment.from_mp3(mp3_path)
        sound.export(wav_path, format="wav")

        # Add the converted filename to the set
        converted_files.add(mp3_file)

# Save the updated set of converted filenames
with open(converted_file_path, 'w') as file:
    file.write('\n'.join(converted_files))

print("Conversion complete. WAV files are saved in:", destination_directory)


Conversion complete. WAV files are saved in: /content/drive/MyDrive/WAV_Files_Recordings/hindi


In [ ]:
#Removing the Silences

# Specify the source directory containing your input WAV files
source_directory = "/content/drive/MyDrive/WAV_Files_Recordings/turkish"

# Specify the destination directory for the processed WAV files
destination_directory = "/content/drive/MyDrive/Removed_Pauses/turkish"

# Specify the log file to store the record of processed files
log_file = "/content/drive/MyDrive/TXT_Files_AccentConversions/removingPauses_log.txt"

# Ensure the destination directory exists, or create it
os.makedirs(destination_directory, exist_ok=True)

# Function to remove silence from audio files
def remove_silence(input_file, output_file):
    audio = AudioSegment.from_wav(input_file)

    # Split on silence with a threshold of -50 dBFS (adjust as needed)
    audio_segments = split_on_silence(audio, silence_thresh=-50)

    # Concatenate non-silent segments
    processed_audio = sum(audio_segments)

    # Export the processed audio to a new WAV file
    processed_audio.export(output_file, format="wav")

# Function to log processed files
def log_processed_file(log_file, input_file, output_file):
    with open(log_file, 'a') as log:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log.write(f"{timestamp} - Processed: {input_file} -> {output_file}\n")

# Loop through the WAV files in the source directory and process them
for wav_file in os.listdir(source_directory):
    if wav_file.endswith(".wav"):
        wav_path = os.path.join(source_directory, wav_file)
        output_path = os.path.join(destination_directory, wav_file)

        # Remove silence from the audio file
        remove_silence(wav_path, output_path)

        # Log the processed file
        log_processed_file(log_file, wav_path, output_path)

print("Silence removal complete. Processed WAV files are saved in:", destination_directory)
print("Processing log is saved in:", log_file)


Silence removal complete. Processed WAV files are saved in: /content/drive/MyDrive/Removed_Pauses/turkish
Processing log is saved in: /content/drive/MyDrive/TXT_Files_AccentConversions/removingPauses_log.txt


In [ ]:
#Making Spectrograms
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np

# Specify the source directory containing your WAV files
source_directory = "/content/drive/MyDrive/Removed_Pauses/turkish"

# Specify the destination directory for saving spectrogram images
destination_directory = "/content/drive/MyDrive/Spectrograms_Accent_Detection/turkish"

# Specify the path for tracking processed files
processed_files_path = "/content/drive/MyDrive/TXT_Files_AccentConversions/spectrograms_log.txt"

# Set the batch size (adjust as needed)
batch_size = 100

# Create or load the list of already processed files
processed_files = set()

if os.path.exists(processed_files_path):
    with open(processed_files_path, "r") as file:
        processed_files.update(file.read().splitlines())

# Function to convert a list of WAV files to spectrogram images
def batch_wav_to_spectrogram(wav_files, destination_directory):
    for wav_file in wav_files:
        if wav_file.endswith(".wav") and wav_file not in processed_files:
            # Load the WAV file
            audio, sr = librosa.load(wav_file)

            # Generate the spectrogram
            spectrogram = np.abs(librosa.stft(audio))

            # Convert the spectrogram to decibels
            spectrogram_db = librosa.amplitude_to_db(spectrogram, ref=np.max)

            # Create a matplotlib figure
            plt.figure(figsize=(10, 5))
            librosa.display.specshow(spectrogram_db, sr=sr, x_axis='time', y_axis='log')

            # Save the spectrogram as an image (e.g., PNG)
            spectrogram_filename = os.path.splitext(os.path.basename(wav_file))[0] + "_spectrogram.png"
            spectrogram_path = os.path.join(destination_directory, spectrogram_filename)
            plt.savefig(spectrogram_path, bbox_inches='tight', pad_inches=0)
            plt.close()

            # Mark the file as processed
            processed_files.add(wav_file)

            # Save the list of processed files
            with open(processed_files_path, "a") as file:
                file.write(wav_file + "\n")

# Get a list of all WAV files in the source directory
wav_files = [os.path.join(source_directory, f) for f in os.listdir(source_directory) if f.endswith(".wav")]

# Process WAV files in batches
for i in range(0, len(wav_files), batch_size):
    batch = wav_files[i:i + batch_size]
    batch_wav_to_spectrogram(batch, destination_directory)

print("Spectrogram conversion complete. Spectrogram images are saved in:", destination_directory)


Spectrogram conversion complete. Spectrogram images are saved in: /content/drive/MyDrive/Spectrograms_Accent_Detection/turkish


In [ ]:
def extract_mfcc(file_path, num_mfcc=13, n_fft=2048, hop_length=512):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    return mfccs

def process_and_save(input_folder, output_folder, log_file):
    processed_files = []

    for filename in os.listdir(input_folder):
        if filename.endswith(".wav"):
            input_file_path = os.path.join(input_folder, filename)
            output_file_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.npy")

            # Check if the file has already been processed
            if os.path.exists(output_file_path):
                print(f"Skipping {input_file_path} - Already processed")
                processed_files.append(output_file_path)
                continue

            mfcc = extract_mfcc(input_file_path)

            # Pad or truncate MFCCs to a fixed length
            if mfcc.shape[1] < 100:
                mfcc = np.pad(mfcc, ((0, 0), (0, 100 - mfcc.shape[1])), mode='constant')
            else:
                mfcc = mfcc[:, :100]

            np.save(output_file_path, mfcc)
            print(f"Processed {input_file_path} and saved to {output_file_path}")
            processed_files.append(output_file_path)

    # Store the list of processed files in a text file
    with open(log_file, 'w') as file:
        for file_path in processed_files:
            file.write(file_path + '\n')

    print(f"\nList of processed files saved to {log_file}")
    return log_file

# Set your input and output folders
input_folder = '/content/drive/MyDrive/Removed_Pauses/turkish'
output_folder = '/content/drive/MyDrive/MFCC_Features/turkish'
log_file = '/content/drive/MyDrive/TXT_Files_AccentConversions/MFCClogfile.txt'

# Process and save files
processed_log_file = process_and_save(input_folder, output_folder, log_file)


Processed /content/drive/MyDrive/Removed_Pauses/turkish/turkish11.wav and saved to /content/drive/MyDrive/MFCC_Features/turkish/turkish11.npy
Processed /content/drive/MyDrive/Removed_Pauses/turkish/turkish12.wav and saved to /content/drive/MyDrive/MFCC_Features/turkish/turkish12.npy
Processed /content/drive/MyDrive/Removed_Pauses/turkish/turkish15.wav and saved to /content/drive/MyDrive/MFCC_Features/turkish/turkish15.npy
Processed /content/drive/MyDrive/Removed_Pauses/turkish/turkish1.wav and saved to /content/drive/MyDrive/MFCC_Features/turkish/turkish1.npy
Processed /content/drive/MyDrive/Removed_Pauses/turkish/turkish14.wav and saved to /content/drive/MyDrive/MFCC_Features/turkish/turkish14.npy
Processed /content/drive/MyDrive/Removed_Pauses/turkish/turkish10.wav and saved to /content/drive/MyDrive/MFCC_Features/turkish/turkish10.npy
Processed /content/drive/MyDrive/Removed_Pauses/turkish/turkish13.wav and saved to /content/drive/MyDrive/MFCC_Features/turkish/turkish13.npy
Processe